In [9]:
#!pip install cenpy
#!pipinstall osmnx
import cenpy
import osmnx
import matplotlib.pyplot as plt

#!pip install contextily
import contextily
import geopandas

acs = cenpy.products.ACS(2019)

In [2]:
# Example: Interacting with cenpy ACS()
# B08006: SEX OF WORKERS BY MEANS OF TRANSPORTATION TO WORK
#sf_df = cenpy.products.ACS().from_place('San Francisco, CA', level='tract', variables=['B08006'])

- [Social Explorer: Census Lookup Table](https://www.socialexplorer.com/data/ACS2019_5yr/metadata/?ds=SE)

In [11]:
vars_to_download = {
    "B01003_001E" :"total_pop", # Total population
    "B08011_001E": "time_leave", #Sex of Workers by Time Leaving Home to Go to Work
    "B08012_001E": "time_travel",  # Sex of Workers by Travel Time to Work
    "B08013_001E": "agg_travel_time", # Aggregate Travel Time to Work (In Minutes) of Workers by Sex
    "B08014_001E": "num_veh", # Sex of Workers by Vehicles Available
    "B08015_001E": "num_veh_work" # Aggregate Number of Vehicles (Car, Truck, or Van) Used in Commuting by Workers 16 Years and Over by Sex
}
vars_to_download_l = list(vars_to_download.keys())


# def getCensusData(place, variables, level='tract'):
#   '''Return a dataframe containing selected census data of a level of region
#   level default is 'tract'. Also allow 'county'.
#   '''
#   db = acs.from_msa(msa=place, level=level, variables=vars_to_download_l)
#   #db = cenpy.products.ACS().from_state(state=place, level='tract', variables=vars_to_download_l)

#   # Change column names back to readable formats
#   db = db.rename(columns=vars_to_download)
#   return db


In [17]:
%%time
db = acs.from_msa("San Francisco, CA",level="tract",variables=vars_to_download_l)
db.head(3)

/opt/anaconda3/envs/skmob21/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda3/envs/skmob21/lib/python3.7/site-packages/cenpy/products.py:749: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  msa, variables, level, "Metropolitan Statistical Area", **kwargs


CPU times: user 2.11 s, sys: 163 ms, total: 2.27 s
Wall time: 1min 3s


,GEOID,geometry,B01003_001E,B08011_001E,B08012_001E,B08013_001E,B08014_001E,B08015_001E,NAME,state,county,tract
0,06081613600,"POLYGON ((-13639358.830 4519608.220, -13639346...",6108.0,2557.0,2557.0,90860.0,2939.0,2100.0,"Census Tract 6136, San Mateo County, California",06,081,613600
1,06041113000,"POLYGON ((-13661535.450 4581804.170, -13661410...",3371.0,1322.0,1322.0,48745.0,1504.0,1175.0,"Census Tract 1130, Marin County, California",06,041,113000
2,06041132100,"POLYGON ((-13663561.130 4568428.830, -13663453...",1825.0,734.0,734.0,24065.0,881.0,595.0,"Census Tract 1321, Marin County, California",06,041,132100


In [18]:
var_names = acs.variables\
               .reindex(vars_to_download)\
               [["label", "concept"]]\
               .reset_index()\
               .rename(columns={"index": "var_id"})
var_names["short_name"] = var_names["var_id"].map(vars_to_download)
var_names

,var_id,label,concept,short_name
0,B01003_001E,Estimate!!Total,TOTAL POPULATION,total_pop
1,B08011_001E,Estimate!!Total:,SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK,time_leave
2,B08012_001E,Estimate!!Total:,SEX OF WORKERS BY TRAVEL TIME TO WORK,time_travel
3,B08013_001E,Estimate!!Aggregate travel time to work (in mi...,AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF ...,agg_travel_time
4,B08014_001E,Estimate!!Total:,SEX OF WORKERS BY VEHICLES AVAILABLE,num_veh
5,B08015_001E,"Estimate!!Aggregate number of vehicles (car, t...","AGGREGATE NUMBER OF VEHICLES (CAR, TRUCK, OR V...",num_veh_work


## Data Cleaning

In [19]:
filler = lambda col: col.fillna(col.mean())
db.loc[:, vars_to_download] = db.loc[:, vars_to_download]\
                                .apply(filler)

In [20]:
# Replace variable codes with short names
db = db.rename(columns=vars_to_download)

In [21]:
db

,GEOID,geometry,total_pop,time_leave,time_travel,agg_travel_time,num_veh,num_veh_work,NAME,state,county,tract
0,06081613600,"POLYGON ((-13639358.830 4519608.220, -13639346...",6108.0,2557.0,2557.0,90860.0,2939.0,2100.0,"Census Tract 6136, San Mateo County, California",06,081,613600
1,06041113000,"POLYGON ((-13661535.450 4581804.170, -13661410...",3371.0,1322.0,1322.0,48745.0,1504.0,1175.0,"Census Tract 1130, Marin County, California",06,041,113000
2,06041132100,"POLYGON ((-13663561.130 4568428.830, -13663453...",1825.0,734.0,734.0,24065.0,881.0,595.0,"Census Tract 1321, Marin County, California",06,041,132100
3,06041114100,"POLYGON ((-13649681.260 4576008.580, -13649545...",5206.0,2584.0,2584.0,81070.0,3108.0,2035.0,"Census Tract 1141, Marin County, California",06,041,114100
4,06041103200,"POLYGON ((-13648029.950 4592966.150, -13648008...",7290.0,3198.0,3198.0,127225.0,3537.0,2560.0,"Census Tract 1032, Marin County, California",06,041,103200
...,...,...,...,...,...,...,...,...,...,...,...,...
975,06013307202,"POLYGON ((-13562865.190 4579978.890, -13562860...",4299.0,1368.0,1368.0,45815.0,1396.0,1025.0,"Census Tract 3072.02, Contra Costa County, Cal...",06,013,307202
976,06013355109,"POLYGON ((-13559344.490 4576952.530, -13559321...",7559.0,3233.0,3233.0,148980.0,3531.0,2460.0,"Census Tract 3551.09, Contra Costa County, Cal...",06,013,355109
977,06001451601,"POLYGON ((-13555645.230 4534831.210, -13555617...",5362.0,2704.0,2704.0,83330.0,2901.0,2365.0,"Census Tract 4516.01, Alameda County, California",06,001,451601
978,06013303102,"POLYGON ((-13550894.450 4575018.250, -13550862...",9625.0,4229.0,4229.0,216545.0,4469.0,3555.0,"Census Tract 3031.02, Contra Costa County, Cal...",06,013,303102


In [22]:
db["area_sqm"] = db.to_crs(epsg=5070).area / 1e6
db.head(3)

,GEOID,geometry,total_pop,time_leave,time_travel,agg_travel_time,num_veh,num_veh_work,NAME,state,county,tract,area_sqm
0,06081613600,"POLYGON ((-13639358.830 4519608.220, -13639346...",6108.0,2557.0,2557.0,90860.0,2939.0,2100.0,"Census Tract 6136, San Mateo County, California",06,081,613600,26.909653
1,06041113000,"POLYGON ((-13661535.450 4581804.170, -13661410...",3371.0,1322.0,1322.0,48745.0,1504.0,1175.0,"Census Tract 1130, Marin County, California",06,041,113000,25.374583
2,06041132100,"POLYGON ((-13663561.130 4568428.830, -13663453...",1825.0,734.0,734.0,24065.0,881.0,595.0,"Census Tract 1321, Marin County, California",06,041,132100,53.619618


In [23]:
db['pct_veh'] = db['num_veh']/(db['total_pop']+(db['total_pop']==0) *1)
db['pct_veh_work'] = db['num_veh_work']/(db['total_pop']+(db['total_pop']==0) *1)
db['avg_travel_time'] = db['agg_travel_time']/(db['total_pop']+(db['total_pop']==0) *1)
db['avg_pop_area'] = db['area_sqm']/(db['total_pop']+(db['total_pop']==0) *1)
db.head(3)

,GEOID,geometry,total_pop,time_leave,time_travel,agg_travel_time,num_veh,num_veh_work,NAME,state,county,tract,area_sqm,pct_veh,pct_veh_work,avg_travel_time,avg_pop_area
0,06081613600,"POLYGON ((-13639358.830 4519608.220, -13639346...",6108.0,2557.0,2557.0,90860.0,2939.0,2100.0,"Census Tract 6136, San Mateo County, California",06,081,613600,26.909653,0.481172,0.343811,14.875573,0.004406
1,06041113000,"POLYGON ((-13661535.450 4581804.170, -13661410...",3371.0,1322.0,1322.0,48745.0,1504.0,1175.0,"Census Tract 1130, Marin County, California",06,041,113000,25.374583,0.446158,0.348561,14.460101,0.007527
2,06041132100,"POLYGON ((-13663561.130 4568428.830, -13663453...",1825.0,734.0,734.0,24065.0,881.0,595.0,"Census Tract 1321, Marin County, California",06,041,132100,53.619618,0.482740,0.326027,13.186301,0.029381


In [24]:
db.to_csv('data/census_clean.csv') #Store df